# Experimento: Atividade 1 - Etapa 4 - Sem DL.

[Proposta On-line](https://docs.google.com/document/d/e/2PACX-1vQQ2A2PZbzVLz7PA0amF7gXN4tMSoGMXbZGpYphJNjSk158CR7jZ9gQWJ12U99xalHuQvSDfF022ypu/pub)

Utilizar modelos pré-treinados de aprendizagem profunda para identificação de segmentos de áudio com deglutições

## Quebrando os arquivos em janelas de 0.5s e passo 0.25s

In [7]:
import pandas as pd
df_ca = pd.read_csv('classificacao_anomalias.csv')
validos = df_ca[df_ca['Label']==0]['File Name'].values
del df_ca
len(validos)

224

In [ ]:
# lendo mp3 e extraindo parâmetros de áudio
import librosa
import numpy as np
from tqdm import tqdm
import os

dados = []
for mp3 in tqdm(validos):
    nome = os.path.basename(mp3)
    yn, sr = librosa.load(mp3)

    janela = 0.5 * sr
    passo = 0.25 * sr

    # janelas de y com tamanho janela e passo passo
    y_janelas = librosa.util.frame(yn, frame_length=int(janela), hop_length=int(passo))

    for y in y_janelas:
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        # fazer a média dos valores
        atributos = [nome]
        atributos.extend(np.mean(chroma_stft,axis=1))
        atributos.extend(np.mean(tonnetz,axis=1))
        atributos.extend(np.mean(rmse,axis=1))
        atributos.extend(np.mean(spec_cent,axis=1))
        atributos.extend(np.mean(spec_bw,axis=1))
        atributos.extend(np.mean(rolloff,axis=1))
        atributos.extend(np.mean(zcr,axis=1))
        atributos.extend(np.mean(mfcc,axis=1))
        atributos.extend(np.mean(melspectrogram,axis=1))

        atributos.extend(np.std(chroma_stft,axis=1))
        atributos.extend(np.std(tonnetz,axis=1))
        atributos.extend(np.std(rmse,axis=1))
        atributos.extend(np.std(spec_cent,axis=1))
        atributos.extend(np.std(spec_bw,axis=1))
        atributos.extend(np.std(rolloff,axis=1))
        atributos.extend(np.std(zcr,axis=1))
        atributos.extend(np.std(mfcc,axis=1))
        atributos.extend(np.std(melspectrogram,axis=1))

        atributos.extend(np.ptp(chroma_stft,axis=1))
        atributos.extend(np.ptp(tonnetz,axis=1))
        atributos.extend(np.ptp(rmse,axis=1))
        atributos.extend(np.ptp(spec_cent,axis=1))
        atributos.extend(np.ptp(spec_bw,axis=1))
        atributos.extend(np.ptp(rolloff,axis=1))
        atributos.extend(np.ptp(zcr,axis=1))
        atributos.extend(np.ptp(mfcc,axis=1))
        atributos.extend(np.ptp(melspectrogram,axis=1))

df = pd.DataFrame(dados)
df.head()